# Лабораторная работа №2  
## Сбор данных и статистика (ipynb)

**Тема:** Прогнозирование цен на продукты питания (временные ряды)

**Цель работы:** выполнить статистический анализ данных о ценах на продукты питания и подготовить данные для обучения модели машинного обучения (нейросети LSTM) с оценкой качества на валидации.

**Задачи работы:**
1. Описать структуру набора данных и проверить его качество.
2. Провести первичный статистический анализ и визуализацию.
3. Подготовить данные для обучения модели (приведение типов, интерполяция до дневного ряда).
4. Обучить модель LSTM и оценить качество (MAE/RMSE).
5. Сравнить качество модели с простым базовым прогнозом (baseline).
6. Сформулировать выводы и ограничения.

**Актуальность данных и допущения:**
- Цены на продукты питания являются социально значимым показателем и отражают инфляционные процессы.
- В учебной версии используется небольшой демонстрационный набор (12 месяцев). Для промышленных применений требуется более длинный и более частотный ряд (например, дневные/недельные наблюдения).
- Преобразование месячных значений в дневной ряд выполняется интерполяцией по времени и является учебным упрощением.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

import tensorflow as tf
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

np.set_printoptions(suppress=True)

## 1. Исходные данные

Набор данных содержит ежемесячные значения цен (в рублях) на основные продукты питания за 2025 год.

Столбцы:
- Milk (1L), Bread (500g), Rice (1kg), Eggs (12), Chicken (1kg), Potatoes (1kg), Apples (1kg)

Далее данные:
- приводятся к типам `float`,
- индексируются по дате,
- визуализируются,
- анализируются статистически,
- подготавливаются для обучения модели прогнозирования.

In [ ]:
RAW_DATA = """Month,Milk (1L),Bread (500g),Rice (1kg),Eggs (12),Chicken (1kg),Potatoes (1kg),Apples (1kg)
2025-01,85,45,75,95,300,40,120
2025-02,87,46,78,97,305,42,125
2025-03,90,48,80,100,310,43,130
2025-04,92,50,82,102,315,44,135
2025-05,95,52,85,105,320,45,140
2025-06,97,54,88,108,325,46,145
2025-07,99,55,90,110,330,47,150
2025-08,100,56,92,112,335,48,155
2025-09,101,57,93,113,340,49,160
2025-10,102,58,95,115,345,50,165
2025-11,103,59,96,116,350,51,170
2025-12,105,60,98,118,355,52,175
"""

df_monthly = pd.read_csv(pd.io.common.StringIO(RAW_DATA))
df_monthly['Month'] = pd.to_datetime(df_monthly['Month'], format='%Y-%m')
df_monthly = df_monthly.set_index('Month').sort_index()
df_monthly = df_monthly.astype(float)

df_monthly.head()

## 2. Проверка качества данных и базовая статистика

In [ ]:
print('Размер:', df_monthly.shape)
print('Пропуски по столбцам:')
display(df_monthly.isna().sum())

display(df_monthly.describe())

## 3. Визуализация месячных рядов

Ниже - динамика цен по всем продуктам на месячной сетке.

In [ ]:
plt.figure(figsize=(12, 5))
for col in df_monthly.columns:
    plt.plot(df_monthly.index, df_monthly[col], marker='o', label=col)
plt.title('Месячная динамика цен по продуктам (2025)')
plt.xlabel('Месяц')
plt.ylabel('Цена, ₽')
plt.grid(True)
plt.legend(loc='best')
plt.show()

## 3.1. Корреляция цен между продуктами

Для дополнительного анализа рассчитаем корреляцию между временными рядами продуктов.
Высокие значения корреляции могут указывать на схожую динамику изменения цен.

Примечание: из-за малого объёма данных (12 точек) интерпретация корреляций ограничена.

In [ ]:
corr = df_monthly.corr()

plt.figure(figsize=(8, 6))
im = plt.imshow(corr.values, aspect='auto')
plt.title('Матрица корреляции (месячные данные)')
plt.xticks(range(len(corr.columns)), corr.columns, rotation=45, ha='right')
plt.yticks(range(len(corr.index)), corr.index)
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.grid(False)
plt.tight_layout()
plt.show()

corr

## 4. Преобразование в дневной ряд (интерполяция)

Почему выполняется интерполяция:
- прогноз в системе строится по дням (пользователь выбирает дату),
- месячных точек недостаточно для последовательной модели на уровне дней,
- интерполяция используется как учебное приближение для построения дневного ряда.

В реальной задаче предпочтительнее использовать исходные дневные/недельные значения цен.

In [ ]:
def monthly_to_daily(df_m: pd.DataFrame) -> pd.DataFrame:
    start = df_m.index.min()
    end = df_m.index.max() + pd.offsets.MonthEnd(0)  # последний день последнего месяца
    daily_index = pd.date_range(start=start, end=end, freq='D')
    df_d = df_m.reindex(daily_index)
    df_d = df_d.interpolate(method='time').ffill().bfill()
    df_d.index.name = 'Дата'
    return df_d

df_daily = monthly_to_daily(df_monthly)
df_daily.tail()

### 4.1. Пример дневного ряда (Milk (1L))

In [ ]:
product = 'Milk (1L)'

plt.figure(figsize=(12, 4))
plt.plot(df_daily.index, df_daily[product], label=product)
plt.title(f'Дневной ряд (интерполяция): {product}')
plt.xlabel('Дата')
plt.ylabel('Цена, ₽')
plt.grid(True)
plt.legend()
plt.show()

## 5. Подготовка данных для обучения модели

Этапы подготовки:
- выбор продукта для обучения (для демонстрации - Milk (1L)),
- масштабирование значений (MinMaxScaler),
- формирование обучающих последовательностей длиной `lookback`,
- разделение на train/val без перемешивания (хронологически).

In [ ]:
def make_sequences(series_scaled: np.ndarray, lookback: int):
    X, y = [], []
    for i in range(lookback, len(series_scaled)):
        X.append(series_scaled[i-lookback:i, 0])
        y.append(series_scaled[i, 0])
    X = np.array(X, dtype=np.float32).reshape(-1, lookback, 1)
    y = np.array(y, dtype=np.float32).reshape(-1, 1)
    return X, y

lookback = 30
series = df_daily[[product]].values.astype(np.float32)

scaler = MinMaxScaler()
scaled = scaler.fit_transform(series)

X, y = make_sequences(scaled, lookback)

split = int(len(X) * 0.85)
X_train, y_train = X[:split], y[:split]
X_val, y_val = X[split:], y[split:]

X_train.shape, X_val.shape

## 6. Построение и обучение LSTM

Модель:
- 2 слоя LSTM,
- Dropout для снижения переобучения,
- Dense(1) для предсказания следующего значения.

Применяется EarlyStopping (по val_loss).

In [ ]:
def build_lstm_model(lookback: int, units: int = 32, dropout: float = 0.2, lr: float = 1e-3):
    model = Sequential([
        Input(shape=(lookback, 1)),
        LSTM(units, return_sequences=True),
        Dropout(dropout),
        LSTM(units, return_sequences=False),
        Dropout(dropout),
        Dense(1),
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='mse')
    return model

tf.random.set_seed(42)
np.random.seed(42)

model = build_lstm_model(lookback=lookback, units=32, dropout=0.2, lr=1e-3)
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=120,
    batch_size=16,
    verbose=0,
    callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
)

len(history.history['loss']), history.history['loss'][-1], history.history['val_loss'][-1]

### 6.1. График обучения (loss/val_loss)

In [ ]:
plt.figure(figsize=(10, 3))
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('Ошибка (MSE) по эпохам')
plt.xlabel('Эпоха')
plt.ylabel('MSE')
plt.grid(True)
plt.legend()
plt.show()

## 7. Оценка качества на валидации (MAE, RMSE)

Метрики рассчитываются в исходном масштабе (в рублях), поэтому:
- предсказания и истинные значения возвращаются из шкалы MinMaxScaler в рубли;
- затем считаются MAE и RMSE.

In [ ]:
y_val_pred = model.predict(X_val, verbose=0)

y_val_true_rub = scaler.inverse_transform(y_val)
y_val_pred_rub = scaler.inverse_transform(y_val_pred)

mae = mean_absolute_error(y_val_true_rub, y_val_pred_rub)
rmse = mean_squared_error(y_val_true_rub, y_val_pred_rub, squared=False)

mae, rmse

### 7.1. Базовая линия (naive baseline)

Базовый прогноз:
- прогноз следующего значения равен последнему значению в окне (последний день).

Сравнение с baseline позволяет понять, превосходит ли модель простейшее решение.

In [ ]:
y_val_baseline_scaled = X_val[:, -1, 0].reshape(-1, 1)
y_val_baseline_rub = scaler.inverse_transform(y_val_baseline_scaled)

mae_bl = mean_absolute_error(y_val_true_rub, y_val_baseline_rub)
rmse_bl = mean_squared_error(y_val_true_rub, y_val_baseline_rub, squared=False)

pd.DataFrame({
    'Метрика': ['MAE', 'RMSE'],
    'LSTM': [mae, rmse],
    'Baseline': [mae_bl, rmse_bl],
})

## 8. Пример прогноза на 30 дней вперёд

Демонстрационный прогноз после последней даты ряда.

In [ ]:
def forecast_next_days(model, scaler, series_raw: np.ndarray, lookback: int, horizon_days: int) -> np.ndarray:
    series_scaled = scaler.transform(series_raw)
    window = series_scaled[-lookback:, 0].astype(np.float32)
    preds_scaled = []

    for _ in range(horizon_days):
        x = window.reshape(1, lookback, 1)
        y_hat = float(model.predict(x, verbose=0)[0, 0])
        preds_scaled.append(y_hat)
        window = np.roll(window, -1)
        window[-1] = y_hat

    preds_scaled = np.array(preds_scaled, dtype=np.float32).reshape(-1, 1)
    preds_raw = scaler.inverse_transform(preds_scaled).reshape(-1)
    return preds_raw

HORIZON_DAYS = 30
forecast = forecast_next_days(model, scaler, series, lookback, HORIZON_DAYS)

last_date = df_daily.index[-1]
forecast_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=HORIZON_DAYS, freq='D')

df_forecast = pd.DataFrame({'Дата': forecast_dates, 'Прогноз, ₽': forecast})
df_forecast.head()

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(df_daily.index, df_daily[product], label='История')
plt.plot(df_forecast['Дата'], df_forecast['Прогноз, ₽'], label='Прогноз (30 дней)')
plt.title(f'{product}: история и прогноз на 30 дней')
plt.xlabel('Дата')
plt.ylabel('Цена, ₽')
plt.grid(True)
plt.legend()
plt.show()

## 9. Выводы и ограничения

**Выводы:**
- Данные имеют структуру временных рядов по нескольким продуктам; визуализация показывает рост цен в 2025 году.
- Выполнен статистический анализ (описательная статистика, визуализация, корреляции).
- Данные подготовлены к обучению модели: масштабирование, формирование последовательностей, выделение валидации.
- Обучена LSTM-модель и рассчитаны MAE/RMSE на валидации.
- Выполнено сравнение с простым baseline-прогнозом.

**Ограничения:**
- Небольшой объём данных (12 месяцев) ограничивает точность и стабильность оценок.
- Интерполяция месячных значений до дневных - учебное упрощение.
- Результаты не являются экономической рекомендацией.

## 10. Список источников

1. Hochreiter S., Schmidhuber J. Long Short-Term Memory. Neural Computation, 1997.
2. Goodfellow I., Bengio Y., Courville A. Deep Learning. MIT Press, 2016.
3. Hyndman R. J., Athanasopoulos G. Forecasting: Principles and Practice. OTexts.
4. Документация TensorFlow: https://www.tensorflow.org.
5. Документация scikit-learn: https://scikit-learn.org.